In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# girder_client docs are here: https://girder.readthedocs.io/en/stable/python-client.html#the-python-client-library
import girder_client

from dotenv import load_dotenv

import metadata_pipeline as mph 
import brain_region_maps as brm

In [3]:
load_dotenv()
gc = girder_client.GirderClient(apiUrl=os.environ.get("dsaApiUrl"))
gc.authenticate(apiKey=os.environ.get("dsaApiToken"))

{'_id': '63bda50af50b104ce17506db'}

In [4]:
NeuroPathDemoImages_Collection='63d98235e08050e0a7a8d75d'

In [5]:
mph.populateMetadata(gc, collectionID=NeuroPathDemoImages_Collection, outputFailed=True)

Updating 63e11992e08050e0a7a98190 (1 of 2230) with: {'npWorking': {'Metadata Error': 'File name E97-60_I_tau_2_.ndpi_small.jpg failed to parse'}}
Updating 63e119dfe08050e0a7a98400 (2 of 2230) with: {'npWorking': {'Metadata Error': 'File name E99-04-10_pTDP_1_.ndpi_small.jpg failed to parse'}}
Updating 63e11473e08050e0a7a952ac (3 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-83_A23_HE_ihp.ndpi_small.jpg failed to parse'}}
Updating 63e11474e08050e0a7a952ae (4 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-83_A18_HE_ihp.ndpi_small.jpg failed to parse'}}
Updating 63e11474e08050e0a7a952b0 (5 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-83_A13_HE_ihp.ndpi_small.jpg failed to parse'}}
Updating 63e11474e08050e0a7a952b2 (6 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-83_A22_HE_ihp.ndpi_small.jpg failed to parse'}}
Updating 63e11474e08050e0a7a952b4 (7 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-83_A15_HE_ihp.

Updating 63e11491e08050e0a7a953e2 (57 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-83_A22_LFB-PAS_ihp.ndpi_small.jpg failed to parse'}}
Updating 63e11491e08050e0a7a953e4 (58 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-83_A25_LFB-PAS_ihp.ndpi_small.jpg failed to parse'}}
Updating 63e11491e08050e0a7a953e6 (59 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-83_A17_LFB-PAS_ihp.ndpi_small.jpg failed to parse'}}
Updating 63e11491e08050e0a7a953e8 (60 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-83_A15_LFB-PAS_ihp.ndpi_small.jpg failed to parse'}}
Updating 63e11491e08050e0a7a953ea (61 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-83_A21_LFB-PAS_ihp.ndpi_small.jpg failed to parse'}}
Updating 63e11492e08050e0a7a953ec (62 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-83_A16_LFB-PAS_ihp.ndpi_small.jpg failed to parse'}}
Updating 63e11492e08050e0a7a953ee (63 of 2230) with: {'npWorking': {'Metadat

Updating 63e114b0e08050e0a7a95532 (114 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-124_A13_US_tau.ndpi_small.jpg failed to parse'}}
Updating 63e114b0e08050e0a7a95534 (115 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-124_A10_US_tau.ndpi_small.jpg failed to parse'}}
Updating 63e114b1e08050e0a7a95536 (116 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-124_A16_US_tau.ndpi_small.jpg failed to parse'}}
Updating 63e114b1e08050e0a7a95538 (117 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-124_A19_US_tau.ndpi_small.jpg failed to parse'}}
Updating 63e114b1e08050e0a7a9553a (118 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-124_A11_US_tau-1-nbm.ndpi_small.jpg failed to parse'}}
Updating 63e114b1e08050e0a7a9553c (119 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-124_A16_US_aSYN-4B12.ndpi_small.jpg failed to parse'}}
Updating 63e114b1e08050e0a7a9553e (120 of 2230) with: {'npWorking': {'Metadata Err

Updating 63e114bce08050e0a7a955b2 (171 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-81_A21_HE.ndpi_small.jpg failed to parse'}}
Updating 63e114bce08050e0a7a955b4 (172 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-81_A15_HE.ndpi_small.jpg failed to parse'}}
Updating 63e114bde08050e0a7a955b6 (173 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-81_A7_HE.ndpi_small.jpg failed to parse'}}
Updating 63e114bde08050e0a7a955b8 (174 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-81_A25_HE.ndpi_small.jpg failed to parse'}}
Updating 63e114bde08050e0a7a955ba (175 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-81_A11_HE.ndpi_small.jpg failed to parse'}}
Updating 63e114bde08050e0a7a955bc (176 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-81_A18_HE.ndpi_small.jpg failed to parse'}}
Updating 63e114bde08050e0a7a955be (177 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-81_A24_HE.ndpi_small.jp

Updating 63e114dee08050e0a7a9571a (228 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-125_A18_HE.ndpi_small.jpg failed to parse'}}
Updating 63e114dee08050e0a7a9571c (229 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-125_A2_HE.ndpi_small.jpg failed to parse'}}
Updating 63e114dfe08050e0a7a9571e (230 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-125_A17_HE.ndpi_small.jpg failed to parse'}}
Updating 63e114dfe08050e0a7a95720 (231 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-125_A13_HE.ndpi_small.jpg failed to parse'}}
Updating 63e114dfe08050e0a7a95722 (232 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-125_A6_US_HE.ndpi_small.jpg failed to parse'}}
Updating 63e114dfe08050e0a7a95724 (233 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-125_A25_HE.ndpi_small.jpg failed to parse'}}
Updating 63e114dfe08050e0a7a95726 (234 of 2230) with: {'npWorking': {'Metadata Error': 'File name E11-125_A10_HE.ndpi

...

Updating 63e132efe08050e0a7aa2fc6 (2035 of 2230) with: {'npWorking': {'Metadata Error': 'File name E13-98_A3_BIELS.svs_small.jpg failed to parse'}}
Updating 63e132f0e08050e0a7aa2fc8 (2036 of 2230) with: {'npWorking': {'Metadata Error': 'File name E13-98_A15_HE.svs_small.jpg failed to parse'}}
Updating 63e132f0e08050e0a7aa2fca (2037 of 2230) with: {'npWorking': {'Metadata Error': 'File name E13-98_A10_TAU.svs_small.jpg failed to parse'}}
Updating 63e132f0e08050e0a7aa2fcc (2038 of 2230) with: {'npWorking': {'Metadata Error': 'File name E13-98_A3_AB.svs_small.jpg failed to parse'}}
Updating 63e132f1e08050e0a7aa2fce (2039 of 2230) with: {'npWorking': {'Metadata Error': 'File name E13-98_A14_HE.svs_small.jpg failed to parse'}}
Updating 63e132f1e08050e0a7aa2fd0 (2040 of 2230) with: {'npWorking': {'Metadata Error': 'File name E13-98_A2_TAU.svs_small.jpg failed to parse'}}
Updating 63e132f1e08050e0a7aa2fd2 (2041 of 2230) with: {'npWorking': {'Metadata Error': 'File name E13-98_A1_PTDP-001.svs_

Updating 63d98653e08050e0a7a8faa9 (2092 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-18_A3_TAU.svs_small.jpg failed to parse'}}
Updating 63d98653e08050e0a7a8faad (2093 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-18_A13_SYN.svs_small.jpg failed to parse'}}
Updating 63d98653e08050e0a7a8faaf (2094 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-18_A5_AB.svs_small.jpg failed to parse'}}
Updating 63d98654e08050e0a7a8fab1 (2095 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-18_A13_TAU.svs_small.jpg failed to parse'}}
Updating 63d98654e08050e0a7a8fab3 (2096 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-18_A1_Ubiq.svs_small.jpg failed to parse'}}
Updating 63d98654e08050e0a7a8fab5 (2097 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-18_A18_SYN.svs_small.jpg failed to parse'}}
Updating 63d98654e08050e0a7a8fab7 (2098 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-18_A2_SYN.svs_smal

Updating 63d98671e08050e0a7a8fd71 (2149 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-27_A19_pTDP.svs_small.jpg failed to parse'}}
Updating 63d98672e08050e0a7a8fd75 (2150 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-27_A1_P62.svs_small.jpg failed to parse'}}
Updating 63d98672e08050e0a7a8fd77 (2151 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-27_A1_pTDP.svs_small.jpg failed to parse'}}
Updating 63d98672e08050e0a7a8fd7b (2152 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-27_A5_pTDP.svs_small.jpg failed to parse'}}
Updating 63d98672e08050e0a7a8fd7d (2153 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-31_A20_HE.svs_small.jpg failed to parse'}}
Updating 63d98672e08050e0a7a8fd83 (2154 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-27_A4_pTDP.svs_small.jpg failed to parse'}}
Updating 63d98673e08050e0a7a8fd87 (2155 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-27_A22_P62.svs_s

Updating 63d98698e08050e0a7a90025 (2206 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-31_A22_PTDP.svs_small.jpg failed to parse'}}
Updating 63d98698e08050e0a7a90027 (2207 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-31_A19_TAU.svs_small.jpg failed to parse'}}
Updating 63d98698e08050e0a7a9002b (2208 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-31_A19_PTDP.svs_small.jpg failed to parse'}}
Updating 63d98698e08050e0a7a9002d (2209 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-31_A18_TAU.svs_small.jpg failed to parse'}}
Updating 63d98698e08050e0a7a9002f (2210 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-31_A2_HE.svs_small.jpg failed to parse'}}
Updating 63d98698e08050e0a7a90031 (2211 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-31_A20_TAU.svs_small.jpg failed to parse'}}
Updating 63d98699e08050e0a7a9003b (2212 of 2230) with: {'npWorking': {'Metadata Error': 'File name E19-31_A5_SYN.svs_s

{'63e12c5ee08050e0a7aa0728': {'name': 'PSTVE_CNTRL_FOR_OS89-44_AB.ndpi_small.jpg',
  'meta': {'npWorking': {'Metadata Error': 'File name PSTVE_CNTRL_FOR_OS89-44_AB.ndpi_small.jpg failed to parse'}}},
 '63e12c5fe08050e0a7aa072a': {'name': 'OS89-44_Ab.ndpi_small.jpg',
  'meta': {'npWorking': {'Metadata Error': 'File name OS89-44_Ab.ndpi_small.jpg failed to parse'}}},
 '63e12c60e08050e0a7aa0730': {'name': 'E39-44_AB.ndpi_small.jpg',
  'meta': {'npWorking': {'Metadata Error': 'File name E39-44_AB.ndpi_small.jpg failed to parse'}}},
 '63e12c60e08050e0a7aa0732': {'name': 'E39-44_TAU.ndpi_small.jpg',
  'meta': {'npWorking': {'Metadata Error': 'File name E39-44_TAU.ndpi_small.jpg failed to parse'}}},
 '63e12c60e08050e0a7aa0734': {'name': 'E39-44_BLANK.ndpi_small.jpg',
  'meta': {'npWorking': {'Metadata Error': 'File name E39-44_BLANK.ndpi_small.jpg failed to parse'}}},
 '63e12c60e08050e0a7aa0736': {'name': 'E39-44_SYN.ndpi_small.jpg',
  'meta': {'npWorking': {'Metadata Error': 'File name E39-4

In [6]:
mph.auditMetadata(gc, collectionID=NeuroPathDemoImages_Collection, outputRecords=True)

In [7]:
mph.getSummaryStats(gc, collectionID=NeuroPathDemoImages_Collection)

Valid Count 32277

Invalid Count: 11847

Control Count: 687


In [8]:
mph.getMissingBrainRegion()